In [9]:
import FINE as fn
import numpy as np
import pandas as pd

In [10]:
numberOfTimeSteps = 4
hoursPerTimeStep = 2190

# Create an energy system model instance 
esM = fn.EnergySystemModel(locations={'ElectrolyzerLocation', 'IndustryLocation'}, 
                            commodities={'electricity', 'hydrogen'}, 
                            numberOfTimeSteps=numberOfTimeSteps,
                            commodityUnitsDict={'electricity': r'kW$_{el}$', 'hydrogen': r'kW$_{H_{2},LHV}$'},
                            hoursPerTimeStep=hoursPerTimeStep, costUnit='1 Euro', 
                            lengthUnit='km', 
                            verboseLogLevel=2)

In [11]:
type(esM)

FINE.energySystemModel.EnergySystemModel

In [4]:
### Buy electricity at the electricity market
costs = pd.DataFrame([np.array([ 0.05, 0., 0.1, 0.051,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
revenues = pd.DataFrame([np.array([ 0., 0.01, 0., 0.,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
maxpurchase = pd.DataFrame([np.array([1e6, 1e6, 1e6, 1e6,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep

In [7]:
maxpurchase

,ElectrolyzerLocation,IndustryLocation
0,2.190000e+09,0.0
1,2.190000e+09,0.0
2,2.190000e+09,0.0
3,2.190000e+09,0.0


In [12]:
fn.Source(esM=esM, name='Electricity market', commodity='electricity', 
                    hasCapacityVariable=False, operationRateMax = maxpurchase,
                    commodityCostTimeSeries = costs,  
                    commodityRevenueTimeSeries = revenues  
                    )

AttributeError: 'NoneType' object has no attribute 'astype'

In [13]:
### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electrolyzers', physicalUnit=r'kW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=500, # euro/kW
                      opexPerCapacity=500*0.025, 
                      interestRate=0.08,
                      economicLifetime=10))

In [14]:
### Hydrogen filled somewhere
esM.add(fn.Storage(esM=esM, name='Pressure tank', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   stateOfChargeMin=0.33, 
                   investPerCapacity=0.5, # eur/kWh
                   interestRate=0.08,
                   economicLifetime=30))

In [15]:
### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines', commodity='hydrogen',
                        hasCapacityVariable=True,
                        investPerCapacity=0.177, 
                        interestRate=0.08, 
                        economicLifetime=40))

In [17]:
### Industry site
demand = pd.DataFrame([np.array([0., 0., 0., 0.,]), np.array([6e3, 6e3, 6e3, 6e3,]),],
                index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Sink(esM=esM, name='Industry site', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix = demand
                ))

AttributeError: 'NoneType' object has no attribute 'columns'